# Analyse Exploratoire du fichier myreport_ping_full.csv

### Imports (fichier & modules)

In [ ]:
import pandas as pda
import matplotlib.pyplot as plt
import numpy as np
#import os
#print(os.getcwd())
#print(os.listdir('241016_LogETL'))

In [ ]:
tab = pda.read_csv('../data/logServer/myreport_ping_full.csv',sep=',')

### Le fichier myreport_ping_full.csv : taille, colonnes

In [ ]:
print("il y a ",tab.shape[0]," lignes et ",tab.shape[1]," colonnes dans le data frame")

In [ ]:
tab.columns

In [ ]:
tab.info()

On peut déjà voir que :
- faudra supprimer les colonnes 8 et 9 car elles sont vides
- colonnes 21, 23, 25, 26 contiennent certains élements vides

In [ ]:
# Calculer le nombre de valeurs uniques pour chaque colonne
nb_valeurs_uniques = tab.nunique()

# Afficher les résultats
print(nb_valeurs_uniques)

En plus des colonnes 8 et 9, on pourra supprimer les colonnes 15, 16, 17. 

In [ ]:
tab.head()


## Regardons le détail par colonne